![title](DEP_LOGO_2.jpg)

# Trend Station Water Quality Data 
## Web Application Demo - Multi-Station

In [1]:
#HIDDEN

import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interactive
import pandas as pd
import csv
from geopy.geocoders import Nominatim
from time import sleep
import folium
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
import seaborn as sns
from scipy import stats
from scipy.stats import linregress
from arcgis.gis import GIS
import geopandas as gpd

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 3000)

[Link to New WVDEP Trend Station Map](https://bonnerjf33.maps.arcgis.com/apps/webappviewer/index.html?id=dc634873ef5e4275b7120482e93a080c)

In [2]:
#HIDDEN

#Filter Warnings!
import warnings
warnings.filterwarnings("ignore")

# Display the associated webpage in a new window
import IPython
url = 'https://bonnerjf33.maps.arcgis.com/apps/webappviewer/index.html?id=dc634873ef5e4275b7120482e93a080c'
iframe = '<iframe src=' + url + ' width=990 height=1000 frameborder=1></iframe>'
IPython.display.HTML(iframe)

## Select Trend Station:

In [3]:
#HIDDEN

def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

df_comb = pd.read_csv('TS_New_Combined_Final.csv')
df_comb = df_comb.fillna(0)

dfhead = pd.read_csv('TS_Locations_Actual.csv')
dfhead = dfhead.fillna(0)
dfheadloc = dfhead[dfhead.latitude != 0]

output_two = widgets.Output()
output_three = widgets.Output()
output_four = widgets.Output()
output_five = widgets.Output()
output_six = widgets.Output()
plot1_output = widgets.Output()
plot_output = widgets.Output()

style = {'description_width': 'initial'}
dropdown_stations = widgets.Dropdown(options = unique_sorted_values(df_comb.station_id), description='Trend Station', style=style)

def common_filtering1(station):
    
    output_two.clear_output()
    output_three.clear_output()
    output_four.clear_output()
    output_five.clear_output()
    output_six.clear_output()
    #plot_output.clear_output()-OUTPUT USED ELSEWHERE!!!
    
    df_comb = pd.read_csv('TS_New_Combined_Final.csv')
    df_comb = df_comb.fillna(0)
    
    common_filter1 = df_comb[(df_comb.station_id == station)]
    
    global ts
    ts = dropdown_stations.value
    
    df_comb = df_comb[df_comb['station_id'].str.contains(str(ts), na=False)]
        
    group = df_comb.groupby('param_name')
    df2 = pd.DataFrame(group.apply(lambda x: x['test_batch_id'].unique()))

    df2.reset_index(inplace=True)
    df2 = df2.rename({0: 'DATES'}, axis='columns')

    dates = df2[df2.param_name == 'LAB CONDUCTIVITY'].DATES.item()

    initial_date = dates[0]
    
    df_comb_date = common_filter1
    df_comb_date = df_comb_date[df_comb_date['test_batch_id'] == initial_date]
    df_comb_date = df_comb_date.drop(['station_id', 'sample_id', 'result_unit', 'test_batch_id'], axis=1)
    df_comb_date = df_comb_date.set_index('param_name').transpose()

    #df_comb_date = df_comb_date.astype('float')-THIS WAS NOT WORKING!!!! UNNECESSARY!!!
    #df_comb_date.columns = ['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB ph', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE']

    global df_comb_concat
    df_comb_concat = []
    df_comb_concat = pd.concat([df_comb_date], axis=0, sort=False)

    as_list = df_comb_concat.index.tolist()
    idx = as_list.index('result_value')
    as_list[idx] = initial_date
    df_comb_concat.index = as_list

    for i, date in enumerate(dates):
    
        df_comb_date = common_filter1
        df_comb_date = df_comb_date[df_comb_date['test_batch_id'] == dates[i]]
        df_comb_date = df_comb_date.drop(['station_id', 'sample_id', 'result_unit', 'test_batch_id'], axis=1)
        df_comb_date = df_comb_date.set_index('param_name').transpose()
    
        #df_comb_date = df_comb_date.astype('float')-THIS WAS NOT WORKING!!!! UNNECESSARY!!!

        #df_comb_date.columns = ['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB pH', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE']
    
        df_comb_concat = pd.concat([df_comb_concat, df_comb_date], axis=0, sort=False)
    
        as_list = df_comb_concat.index.tolist()
        idx = as_list.index('result_value')
        as_list[idx] = dates[i]
        df_comb_concat.index = as_list

        df_comb_concat.index.name = str(dropdown_stations.value)
        
    #df_comb_concat = df_comb_concat.fillna(0)
    
    #df_comb_concat = df_comb_concat.loc[:, (df_comb_date != 0).any(axis=0)]-THIS WAS DELETING COLUMNS WITH ANY ZERO VALUES!!!

    df_comb_concat.index = pd.to_datetime(df_comb_concat.index)

    df_comb_concat = df_comb_concat.iloc[1:]
    
    #global df_comb_concat_old
    df_comb_concat_old = pd.read_csv('TS_Old_Final/' + ts + '_Old_Combined_Final.csv')
    df_comb_concat_old.set_index(str(ts), inplace=True)
    df_comb_concat_old.index = pd.to_datetime(df_comb_concat_old.index)
    df_comb_concat_old.columns = ['FIELD ph', 'FIELD CONDUCTIVITY', 'TEMPERATURE', 'FIELD DISSOLVED OXYGEN', 'FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB pH', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T']
    df_comb_concat_old.replace('', 0, inplace=True)
    df_comb_concat_old.replace('FROZEN', 0, inplace=True)
    df_comb_concat_old['FLOW'].fillna(0, inplace=True)
    df_comb_concat_old['FLOW'] = df_comb_concat_old['FLOW'].astype('float')
    pd.to_numeric(df_comb_concat_old['FLOW'])
    
    df_comb_concat = pd.concat([df_comb_concat_old, df_comb_concat], axis=0, sort=False)
      
    #global dfdesc
    #dfdesc = df_comb_concat.describe()
    #dfdesc.index.name = str(ts)

    #global dfcorr
    #dfcorr = df_comb_concat.corr()
    #dfcorr = dfcorr.dropna(axis=0, how='all') #Resolves problem from above step
    #dfcorr = dfcorr.dropna(axis=1, how='all') #Resolves problem from above step
    #dfcorr.index.name = str(ts)
    
    global common_filter
    common_filter = dfheadloc[(dfheadloc.station_id == station)]    
    
    with output_two:
        display(common_filter)
    
    with output_three:
        display(common_filter1)
        
    #with output_four:
        #display(df_comb_concat)
    
    #with output_five:
        #display(dfdesc)
        
    with output_six:
        display(dfcorr)
              
def dropdown_stations_eventhandler(change):
    common_filtering1(change.new)
    
dropdown_stations.observe(dropdown_stations_eventhandler, names='value')  
  
display(dropdown_stations)

Dropdown(description='Trend Station', options=('TS001', 'TS002', 'TS003', 'TS004', 'TS005', 'TS006', 'TS007', …

## Trend Station Location Information:

In [4]:
#HIDDEN

display(output_two)

Output()

## Water Quality Data Plots:

In [5]:
#HIDDEN

def unique_sorted_values_plus_none(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, 'NONE')
    return unique

labels=pd.Series(['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB pH', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE'])

dropdown_param = widgets.Dropdown(options = unique_sorted_values(labels), description='Parameter 1')
dropdown_param3 = widgets.Dropdown(options = unique_sorted_values_plus_none(labels), description='Parameter 2')
button1 = widgets.Button(description='Display/Update Plot')

style = {'description_width': 'initial'}

year_slider = widgets.IntRangeSlider(value=[15, 18],min=2,max=18,step=1,layout=widgets.Layout(width='60%', height='20px'),description='Data Range: (Total: 2002 to 2018, Default: 2015 to 2018)',style=style,disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d')

plot1_output = widgets.Output()

def on_button_click(param, param3, year1, year2):
        plot1_output.clear_output()
        box2.value = False
        box3.value = False
        box4.value = False
        global df_comb_concat_filt
        with plot1_output:
            if param not in df_comb_concat.columns:
                print('NO DATA EXISTS FOR THE CHOSEN STATION/PARAMETER COMBINATION! PLEASE SELECT ANOTHER PARAMETER.')
    
            elif param3 == 'NONE' and param in df_comb_concat.columns:
                year1 = 2000 + int(year_slider.value[0])
                year2 = 2000 + int(year_slider.value[1])
                plt.figure(figsize=(20,8));
                df_comb_concat_filt = df_comb_concat[str(year1):str(year2)]
                plt.plot(df_comb_concat_filt[param], 'go--', linewidth=1, markersize=6);
                plt.grid();
                color1 = 'tab:green'
                plt.tick_params(axis='y', labelcolor=color1)
                plt.ylabel(param, fontsize=15, color=color1);
                plt.xlabel('Date', fontsize=15);
                plt.title(str(ts), fontsize=25);
                plt.show();
                
            
            elif param in df_comb_concat and param3 not in df_comb_concat.columns:
                year1 = 2000 + int(year_slider.value[0])
                year2 = 2000 + int(year_slider.value[1])
                plt.figure(figsize=(20,8));
                df_comb_concat_filt = df_comb_concat[str(year1):str(year2)]
                plt.plot(df_comb_concat_filt[param], 'go--', linewidth=1, markersize=6);
                plt.grid();
                color1 = 'tab:green'
                plt.tick_params(axis='y', labelcolor=color1)
                plt.ylabel(param, fontsize=15, color=color1);
                plt.xlabel('Date', fontsize=15);
                plt.title(str(ts), fontsize=25);
                plt.show();
                print('NO DATA EXISTS FOR THE SECOND SELECTED PARAMETER! PLEASE SELECT ANOTHER PARAMETER.')
                
            
            else:
                year1 = 2000 + int(year_slider.value[0])
                year2 = 2000 + int(year_slider.value[1])
                plt.figure(figsize=(20,8));
                df_comb_concat_filt = df_comb_concat[str(year1):str(year2)]
                plt.plot(df_comb_concat_filt[param], 'go--', linewidth=1, markersize=6);
                plt.grid();
                color1 = 'tab:green'
                plt.tick_params(axis='y', labelcolor=color1)
                plt.ylabel(param, fontsize=15, color=color1);
                plt.ylabel(param, fontsize=15);
                plt.xlabel('Date', fontsize=15);
                plt.title(str(ts), fontsize=25);
                ax2 = plt.twinx()  # instantiate a second axes that shares the same x-axis
                color = 'tab:blue'
                ax2.set_ylabel(param3, color=color, fontsize=15)  
                ax2.plot(df_comb_concat_filt[param3], 'bo--', linewidth=1, markersize=6, color=color)
                ax2.tick_params(axis='y', labelcolor=color)
                plt.show();
                
        
def button1_eventhandler(change):
    on_button_click(dropdown_param.value, dropdown_param3.value, year_slider.value[0], year_slider.value[1])

button1.on_click(button1_eventhandler)


def drop_eventhandler2(change):
    plot1_output.clear_output()
    
dropdown_stations.observe(drop_eventhandler2, names='value')

display(dropdown_param)
display(dropdown_param3)
display(year_slider)
display(button1)

Dropdown(description='Parameter 1', options=('ALKALINITY', 'ALUMINUM_D', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', …

Dropdown(description='Parameter 2', options=('NONE', 'ALKALINITY', 'ALUMINUM_D', 'ALUMINUM_T', 'ANTIMONY', 'AR…

IntRangeSlider(value=(15, 18), continuous_update=False, description='Data Range: (Total: 2002 to 2018, Default…

Button(description='Display/Update Plot', style=ButtonStyle())

In [6]:
#HIDDEN

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

display(plot1_output)

Output()

## Water Quality Data:

In [7]:
#HIDDEN

box2 = widgets.Checkbox(False, description='Display Water Quality Data')

data_output = widgets.Output()

def changed1(ts, c):
    data_output.clear_output()
    with data_output:
        if c==True:
            display(df_comb_concat_filt)
        else:
            data_output.clear_output()

def checkbox_eventhandler(change):
    changed1(dropdown_stations.value, change.new)

def checkbox_eventhandler2(change):
    box2.value = False
    data_output.clear_output()
    
box2.observe(checkbox_eventhandler, names='value')
dropdown_stations.observe(checkbox_eventhandler2, names='value')

display(box2)
display(data_output)

Checkbox(value=False, description='Display Water Quality Data')

Output()

## Water Quality Statistics:

In [8]:
#HIDDEN

box3 = widgets.Checkbox(False, description='Display Water Quality Statistics')

data1_output = widgets.Output()

def changed2(ts, d):
    data1_output.clear_output()
    with data1_output:
        if d==True:
            global dfdesc
            dfdesc = df_comb_concat_filt.describe()
            dfdesc.index.name = str(ts)
            display(dfdesc)
        else:
            data1_output.clear_output()

def checkbox_eventhandler(change):
    changed2(dropdown_stations.value, change.new)

def checkbox_eventhandler2(change):
    box3.value = False
    data1_output.clear_output()

box3.observe(checkbox_eventhandler, names='value')
dropdown_stations.observe(checkbox_eventhandler2, names='value')


display(box3)
display(data1_output)

Checkbox(value=False, description='Display Water Quality Statistics')

Output()

## Water Quality Parameter Correlation:

In [9]:
#HIDDEN

box4 = widgets.Checkbox(False, description='Display Pearson R Values')

data2_output = widgets.Output()

def changed3(ts, e):
    data2_output.clear_output()
    with data2_output:
        if e==True:
            global dfcorr
            dfcorr = df_comb_concat_filt.corr()
            dfcorr = dfcorr.dropna(axis=0, how='all') #Resolves problem from above step
            dfcorr = dfcorr.dropna(axis=1, how='all') #Resolves problem from above step
            dfcorr.index.name = str(ts)
            display(dfcorr)
        else:
            data2_output.clear_output()

def checkbox_eventhandler(change):
    changed3(dropdown_stations.value, change.new)

def checkbox_eventhandler2(change):
    box4.value = False
    data2_output.clear_output()
    
box4.observe(checkbox_eventhandler, names='value')
dropdown_stations.observe(checkbox_eventhandler2, names='value')

display(box4)
display(data2_output)

Checkbox(value=False, description='Display Pearson R Values')

Output()

## Water Quality Parameter Comparison by Linear Regression:

In [10]:
#HIDDEN

labels=pd.Series(['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB pH', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE'])

dropdown_param1 = widgets.Dropdown(options = unique_sorted_values(labels), description='Parameter: X')
dropdown_param2 = widgets.Dropdown(options = unique_sorted_values(labels), description='Parameter: Y')
button2 = widgets.Button(description='Display/Update Plot')

plot2_output = widgets.Output()

def on_button_click2(param1, param2):
    plot2_output.clear_output()
    
    with plot2_output: 
        
        if param1 not in df_comb_concat_filt.columns:
            print('NO DATA EXISTS FOR THE CHOSEN STATION/PARAMETER COMBINATION! PLEASE SELECT ANOTHER PARAMETER.')
        elif param2 not in df_comb_concat.columns:
            print('NO DATA EXISTS FOR THE CHOSEN STATION/PARAMETER COMBINATION! PLEASE SELECT ANOTHER PARAMETER.')
        else:
            width = 20;
            height = 8;
            plt.figure(figsize=(width, height));
            plt.ylabel(param2, fontsize=15);
            plt.xlabel(param1, fontsize=15);
            sns.regplot(df_comb_concat_filt[param1], df_comb_concat_filt[param2], color='green').set_title(ts, fontsize=20);
            plt.show()
            #pearson_coef, p_value = stats.pearsonr(df_comb_concat[param1], df_comb_concat[param2]) 
            
            #stats.pearsonr(df_comb_concat[param1], df_comb_concat[param2]);
            
            #print("The Pearson Correlation Coefficient is", pearson_coef, " with a p-value of", p_value, " for parameters", param1, "(X) and", param2, "(Y)");  
            
def button4_eventhandler(change):
    plot2_output.clear_output()

button1.on_click(button4_eventhandler)

def button2_eventhandler(change):
    on_button_click2(dropdown_param1.value, dropdown_param2.value)

button2.on_click(button2_eventhandler)

def drop_eventhandler3(change):
    plot2_output.clear_output()
    
dropdown_stations.observe(drop_eventhandler3, names='value')

display(dropdown_param1)
display(dropdown_param2)
display(button2)

Dropdown(description='Parameter: X', options=('ALKALINITY', 'ALUMINUM_D', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC',…

Dropdown(description='Parameter: Y', options=('ALKALINITY', 'ALUMINUM_D', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC',…

Button(description='Display/Update Plot', style=ButtonStyle())

In [11]:
#HIDDEN

display(plot2_output)

Output()

## Select Second Trend Station:

In [12]:
#HIDDEN

def unique_sorted_values5(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

df_comb2 = pd.read_csv('TS_New_Combined_Final.csv')
df_comb2 = df_comb2.fillna(0)

dfhead2 = pd.read_csv('TS_Locations_Actual.csv')
dfhead2 = dfhead2.fillna(0)
dfheadloc2 = dfhead2[dfhead2.latitude != 0]

output_twelve = widgets.Output()
output_thirteen = widgets.Output()
#output_four = widgets.Output()
#output_five = widgets.Output()
output_sixteen = widgets.Output()
#plot1_output = widgets.Output()
#plot_output = widgets.Output()

style = {'description_width': 'initial'}
dropdown_stations2 = widgets.Dropdown(options = unique_sorted_values5(df_comb2.station_id), description='Additional Trend Station', style=style)

def common_filtering2(station):
    
    output_twelve.clear_output()
    output_thirteen.clear_output()
    #output_four.clear_output()
    #output_five.clear_output()
    output_sixteen.clear_output()
    #plot_output.clear_output()-OUTPUT USED ELSEWHERE!!!
    
    df_comb2 = pd.read_csv('TS_New_Combined_Final.csv')
    df_comb2 = df_comb2.fillna(0)
    
    common_filter9 = df_comb2[(df_comb2.station_id == station)]
    
    global ts2
    ts2 = dropdown_stations2.value
    
    df_comb2 = df_comb2[df_comb2['station_id'].str.contains(str(ts2), na=False)]
        
    group = df_comb2.groupby('param_name')
    df3 = pd.DataFrame(group.apply(lambda x: x['test_batch_id'].unique()))

    df3.reset_index(inplace=True)
    df3 = df3.rename({0: 'DATES'}, axis='columns')

    dates2 = df3[df3.param_name == 'LAB CONDUCTIVITY'].DATES.item()

    initial_date2 = dates2[0]
    
    df_comb_date2 = common_filter9
    df_comb_date2 = df_comb_date2[df_comb_date2['test_batch_id'] == initial_date2]
    df_comb_date2 = df_comb_date2.drop(['station_id', 'sample_id', 'result_unit', 'test_batch_id'], axis=1)
    df_comb_date2 = df_comb_date2.set_index('param_name').transpose()

    #df_comb_date = df_comb_date.astype('float')-THIS WAS NOT WORKING!!!! UNNECESSARY!!!
    #df_comb_date.columns = ['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB ph', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE']

    global df_comb_concat2
    df_comb_concat2 = []
    df_comb_concat2 = pd.concat([df_comb_date2], axis=0, sort=False)

    as_list2 = df_comb_concat2.index.tolist()
    idx2 = as_list2.index('result_value')
    as_list2[idx2] = initial_date2
    df_comb_concat2.index = as_list2

    for i, date in enumerate(dates2):
    
        df_comb_date2 = common_filter9
        df_comb_date2 = df_comb_date2[df_comb_date2['test_batch_id'] == dates2[i]]
        df_comb_date2 = df_comb_date2.drop(['station_id', 'sample_id', 'result_unit', 'test_batch_id'], axis=1)
        df_comb_date2 = df_comb_date2.set_index('param_name').transpose()
    
        #df_comb_date = df_comb_date.astype('float')-THIS WAS NOT WORKING!!!! UNNECESSARY!!!

        #df_comb_date.columns = ['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB pH', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE']
    
        df_comb_concat2 = pd.concat([df_comb_concat2, df_comb_date2], axis=0, sort=False)
    
        as_list2 = df_comb_concat2.index.tolist()
        idx2 = as_list2.index('result_value')
        as_list2[idx2] = dates2[i]
        df_comb_concat2.index = as_list2

        df_comb_concat2.index.name = str(dropdown_stations2.value)
        
    #df_comb_concat = df_comb_concat.fillna(0)
    
    #df_comb_concat = df_comb_concat.loc[:, (df_comb_date != 0).any(axis=0)]-THIS WAS DELETING COLUMNS WITH ANY ZERO VALUES!!!

    df_comb_concat2.index = pd.to_datetime(df_comb_concat2.index)

    df_comb_concat2 = df_comb_concat2.iloc[1:]
    
    #global df_comb_concat_old
    df_comb_concat_old2 = pd.read_csv('TS_Old_Final/' + ts2 + '_Old_Combined_Final.csv')
    df_comb_concat_old2.set_index(str(ts2), inplace=True)
    df_comb_concat_old2.index = pd.to_datetime(df_comb_concat_old2.index)
    df_comb_concat_old2.columns = ['FIELD ph', 'FIELD CONDUCTIVITY', 'TEMPERATURE', 'FIELD DISSOLVED OXYGEN', 'FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB pH', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T']
    df_comb_concat_old2.replace('', 0, inplace=True)
    df_comb_concat_old2.replace('FROZEN', 0, inplace=True)
    df_comb_concat_old2['FLOW'].fillna(0, inplace=True)
    df_comb_concat_old2['FLOW'] = df_comb_concat_old2['FLOW'].astype('float')
    pd.to_numeric(df_comb_concat_old2['FLOW'])
    
    df_comb_concat2 = pd.concat([df_comb_concat_old2, df_comb_concat2], axis=0, sort=False)
      
    #global dfdesc
    #dfdesc = df_comb_concat.describe()
    #dfdesc.index.name = str(ts)

    #global dfcorr
    #dfcorr = df_comb_concat.corr()
    #dfcorr = dfcorr.dropna(axis=0, how='all') #Resolves problem from above step
    #dfcorr = dfcorr.dropna(axis=1, how='all') #Resolves problem from above step
    #dfcorr.index.name = str(ts)
    
    global common_filter8
    common_filter8 = dfheadloc2[(dfheadloc2.station_id == station)]    
    
    with output_twelve:
        display(common_filter8)
    
    with output_thirteen:
        display(common_filter9)
        
    #with output_four:
        #display(df_comb_concat)
    
    #with output_five:
        #display(dfdesc)
        
    with output_sixteen:
        display(dfcorr2)
              
def dropdown_stations_eventhandler12(change):
    common_filtering2(change.new)
    
dropdown_stations2.observe(dropdown_stations_eventhandler12, names='value')  
  
display(dropdown_stations2)

Dropdown(description='Additional Trend Station', options=('TS001', 'TS002', 'TS003', 'TS004', 'TS005', 'TS006'…

## Additional Trend Station Location Information:

In [13]:
#HIDDEN

display(output_twelve)

Output()

## Additional Water Quality Data Plots:

In [14]:
#HIDDEN

def unique_sorted_values_plus_none2(array):
    unique2 = array.unique().tolist()
    unique2.sort()
    unique2.insert(0, 'NONE')
    return unique2

labels=pd.Series(['FIELD ph', 'FIELD CONDUCTIVITY','TEMPERATURE','FLOW', 'LAB CONDUCTIVITY', 'SULFATE', 'DISSOLVED SOLIDS', 'SUSPENDED SOLIDS', 'LAB pH', 'ALKALINITY', 'HOT ACIDITY', 'MINERAL ACIDITY', 'IRON_D', 'MANGANESE_D', 'ALUMINUM_D', 'IRON_T', 'MANGANESE_T', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', 'BERYLLIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'LEAD', 'MERCURY', 'NICKEL', 'SELENIUM', 'SILVER', 'THALLIUM', 'ZINC', 'CYANIDE', 'PHENOLS', 'NITRATE', 'SODIUM', 'CALCIUM', 'MAGNESIUM', 'POTASSIUM', 'BI-CARBONATE', 'CARBONATE', 'PHOSPHATE'])

dropdown_param12 = widgets.Dropdown(options = unique_sorted_values(labels), description='Parameter 1')
dropdown_param13 = widgets.Dropdown(options = unique_sorted_values_plus_none2(labels), description='Parameter 2')
button12 = widgets.Button(description='Display/Update Plot')

style = {'description_width': 'initial'}

year_slider2 = widgets.IntRangeSlider(value=[15, 18],min=2,max=18,step=1,layout=widgets.Layout(width='60%', height='20px'),description='Data Range: (Total: 2002 to 2018, Default: 2015 to 2018)',style=style,disabled=False,continuous_update=False,orientation='horizontal',readout=True,readout_format='d')

plot1_output2 = widgets.Output()

def on_button_click12(param12, param13, year12, year13):
        plot1_output2.clear_output()
        box12.value = False
        box13.value = False
        #box14.value = False
        global df_comb_concat_filt2
        with plot1_output2:
            if param12 not in df_comb_concat2.columns:
                print('NO DATA EXISTS FOR THE CHOSEN STATION/PARAMETER COMBINATION! PLEASE SELECT ANOTHER PARAMETER.')
    
            elif param13 == 'NONE' and param12 in df_comb_concat2.columns:
                year12 = 2000 + int(year_slider2.value[0])
                year13 = 2000 + int(year_slider2.value[1])
                plt.figure(figsize=(20,8));
                df_comb_concat_filt2 = df_comb_concat2[str(year12):str(year13)]
                plt.plot(df_comb_concat_filt2[param12], 'go--', linewidth=1, markersize=6);
                plt.grid();
                color1 = 'tab:green'
                plt.tick_params(axis='y', labelcolor=color1)
                plt.ylabel(param12, fontsize=15, color=color1);
                plt.xlabel('Date', fontsize=15);
                plt.title(str(ts2), fontsize=25);
                plt.show();
                
            
            elif param12 in df_comb_concat2 and param13 not in df_comb_concat2.columns:
                year12 = 2000 + int(year_slider2.value[0])
                year13 = 2000 + int(year_slider2.value[1])
                plt.figure(figsize=(20,8));
                df_comb_concat_filt2 = df_comb_concat2[str(year12):str(year13)]
                plt.plot(df_comb_concat_filt2[param12], 'go--', linewidth=1, markersize=6);
                plt.grid();
                color1 = 'tab:green'
                plt.tick_params(axis='y', labelcolor=color1)
                plt.ylabel(param12, fontsize=15, color=color1);
                plt.xlabel('Date', fontsize=15);
                plt.title(str(ts2), fontsize=25);
                plt.show();
                print('NO DATA EXISTS FOR THE SECOND SELECTED PARAMETER! PLEASE SELECT ANOTHER PARAMETER.')
                
            
            else:
                year12 = 2000 + int(year_slider2.value[0])
                year13 = 2000 + int(year_slider2.value[1])
                plt.figure(figsize=(20,8));
                df_comb_concat_filt2 = df_comb_concat2[str(year12):str(year13)]
                plt.plot(df_comb_concat_filt2[param12], 'go--', linewidth=1, markersize=6);
                plt.grid();
                color1 = 'tab:green'
                plt.tick_params(axis='y', labelcolor=color1)
                plt.ylabel(param12, fontsize=15, color=color1);
                plt.ylabel(param12, fontsize=15);
                plt.xlabel('Date', fontsize=15);
                plt.title(str(ts2), fontsize=25);
                ax2 = plt.twinx()  # instantiate a second axes that shares the same x-axis
                color = 'tab:blue'
                ax2.set_ylabel(param13, color=color, fontsize=15)  
                ax2.plot(df_comb_concat_filt2[param13], 'bo--', linewidth=1, markersize=6, color=color)
                ax2.tick_params(axis='y', labelcolor=color)
                plt.show();
                
        
def button12_eventhandler(change):
    on_button_click12(dropdown_param12.value, dropdown_param13.value, year_slider2.value[0], year_slider2.value[1])

button12.on_click(button12_eventhandler)


def drop_eventhandler16(change):
    plot1_output2.clear_output()
    
dropdown_stations2.observe(drop_eventhandler16, names='value')

display(dropdown_param12)
display(dropdown_param13)
display(year_slider2)
display(button12)

Dropdown(description='Parameter 1', options=('ALKALINITY', 'ALUMINUM_D', 'ALUMINUM_T', 'ANTIMONY', 'ARSENIC', …

Dropdown(description='Parameter 2', options=('NONE', 'ALKALINITY', 'ALUMINUM_D', 'ALUMINUM_T', 'ANTIMONY', 'AR…

IntRangeSlider(value=(15, 18), continuous_update=False, description='Data Range: (Total: 2002 to 2018, Default…

Button(description='Display/Update Plot', style=ButtonStyle())

In [15]:
#HIDDEN

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

display(plot1_output2)

Output()

In [16]:
#HIDDEN

box12 = widgets.Checkbox(False, description='Display Additional Water Quality Data', layout=widgets.Layout(width='60%', height='20px'))

data_output12 = widgets.Output()

def changed12(ts2, g):
    data_output12.clear_output()
    with data_output12:
        if g==True:
            display(df_comb_concat_filt2)
        else:
            data_output12.clear_output()

def checkbox_eventhandler12(change):
    changed12(dropdown_stations2.value, change.new)

def checkbox_eventhandler13(change):
    box12.value = False
    data_output12.clear_output()
    
box12.observe(checkbox_eventhandler12, names='value')
dropdown_stations2.observe(checkbox_eventhandler12, names='value')

display(box12)
display(data_output12)

Checkbox(value=False, description='Display Additional Water Quality Data', layout=Layout(height='20px', width=…

Output()

## Additional Water Quality Statistics:

In [17]:
#HIDDEN

box13 = widgets.Checkbox(False, description='Display Additional Water Quality Statistics', layout=widgets.Layout(width='60%', height='20px'))

data1_output12 = widgets.Output()

def changed13(ts, d):
    data1_output.clear_output()
    with data1_output12:
        if d==True:
            global dfdesc2
            dfdesc2 = df_comb_concat_filt2.describe()
            dfdesc2.index.name = str(ts2)
            display(dfdesc2)
        else:
            data1_output12.clear_output()

def checkbox_eventhandler14(change):
    changed13(dropdown_stations2.value, change.new)

def checkbox_eventhandler15(change):
    box13.value = False
    data1_output12.clear_output()

box13.observe(checkbox_eventhandler14, names='value')
dropdown_stations2.observe(checkbox_eventhandler15, names='value')


display(box13)
display(data1_output12)

Checkbox(value=False, description='Display Additional Water Quality Statistics', layout=Layout(height='20px', …

Output()